<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/zindi/landslide_prevention/notebooks/03_lgbm_direct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade optuna
!pip install --upgrade lightgbm

In [2]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

optuna.logging.set_verbosity(optuna.logging.INFO)

import lightgbm as lgb
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [3]:
#remove cell to run future versions
assert optuna.__version__ == '3.0.2', f'Change in Optuna version. Original notebook version: 3.0.2'
assert lgb.__version__ == '3.3.2', f'Change in LightGBM version. Original notebook version: 3.3.2'

In [4]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    HAVE_GPU = True
except Exception:
    HAVE_GPU = False

print(f'GPU available: {HAVE_GPU}')

GPU available: False


In [5]:
SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Data Preparation

In [7]:
DATA_PATH = '/content/drive/MyDrive/data_science_competitions/zindi/landslide_prevention/data'

train = pd.read_csv(f'{DATA_PATH}/raw/train.csv')
test = pd.read_csv(f'{DATA_PATH}/raw/test.csv')
sample_sub = pd.read_csv(f'{DATA_PATH}/raw/sample_sub.csv')

train_agg = pd.read_csv(f'{DATA_PATH}/processed/train_agg.csv')
test_agg = pd.read_csv(f'{DATA_PATH}/processed/test_agg.csv')

In [8]:
TARGET = train['Label']

In [9]:
cat_features = [f for f in test.columns if f.endswith('geology')]
cat_features.extend(['agg_geology_mode', 'agg_geology_nunique'])

In [10]:
def preprocess(df: pd.DataFrame, is_train: bool = False) -> pd.DataFrame:
    
    df = df.drop('Sample_ID', axis=1)
    if is_train:
        df = df.drop('Label', axis=1)
    
    #reduce memory usage
    def reduce_mem(df: pd.DataFrame) -> pd.DataFrame:
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in ['int16', 'int32', 'int64', 'float32', 'float64']:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type).startswith('int'):
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        return df

    df = reduce_mem(df)

    return df

In [11]:
train = preprocess(train, is_train=True)
test = preprocess(test)

In [12]:
train_agg = preprocess(train_agg, is_train=True)
test_agg = preprocess(test_agg)

In [13]:
gc.collect()

22

# Baseline

In [14]:
def custom_f1(y_true, y_pred):
    eval_name = 'custom_f1'
    eval_result = f1_score(y_true, (y_pred >= 0.5).astype('int'))
    is_higher_better = True
    return (eval_name, eval_result, is_higher_better)

In [15]:
%%time
scores_f1 = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
X, y = train, TARGET
feature_name = list(X.columns)
categorical_feature = [f for f in feature_name if f in cat_features]
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = LGBMClassifier(
        objective='binary',
        boosting_type='goss',
        device_type='cpu',
        random_state=SEED
    ) 
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=100,
        eval_metric=custom_f1,
        feature_name=feature_name,
        categorical_feature=categorical_feature,
        verbose=0
    )
    val_preds = model.predict(X_val)

    score = f1_score(y_val, val_preds)
    scores_f1.append(score)
    print(f'Fold #{fold}: ({model.best_iteration_} rounds) F1-score = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg F1-score = {np.mean(scores_f1):.5f} +/- {np.std(scores_f1):.5f}\n')

Fold #0: (91 rounds) F1-score = 0.65538
Fold #1: (98 rounds) F1-score = 0.69360
Fold #2: (98 rounds) F1-score = 0.66867
Fold #3: (79 rounds) F1-score = 0.65680
Fold #4: (64 rounds) F1-score = 0.68177

Avg F1-score = 0.67124 +/- 0.01468

CPU times: user 30.4 s, sys: 201 ms, total: 30.6 s
Wall time: 16.3 s


# Hyperparameter tuning

In [16]:
def objective(trial, data, base_params):

    scores = []
    X, y = data
    feature_name = list(X.columns)
    categorical_feature = [f for f in feature_name if f in cat_features]

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step=0.01),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 200, step=0.1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 200, step=0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 1000, step=5),
        # 'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 0, 500, step=2),
        'min_split_gain': trial.suggest_float('min_split_gain', 0, 15, step=0.01),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.95, step=0.05),
        'top_rate': trial.suggest_float('top_rate', 0.1, 0.5, step=0.05),
        'other_rate': trial.suggest_float('other_rate', 0.05, 0.5, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 5, step=0.05)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = LGBMClassifier(**base_params, **param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric=custom_f1,
            early_stopping_rounds=100,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            verbose=False
        )
        val_preds = model.predict(X_val)
        scores.append(f1_score(y_val, val_preds))
    
    return np.mean(scores)

In [17]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True
    )
    
    return study

# Cross-validation

In [18]:
def evaluate_model(data, model_params, verbose=True):
    oof_preds = {}  #out-of-fold predictions on train set
    test_preds = {} #predictions on test set for each fold
    scores_f1 = [] #F1 scores on validation set

    X, X_test, y = data
    feature_name = list(X.columns)
    categorical_feature = [f for f in feature_name if f in cat_features]

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = LGBMClassifier(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric=custom_f1,
            early_stopping_rounds=100,
            feature_name=feature_name,
            categorical_feature=categorical_feature,
            verbose=False
        )
        val_preds = model.predict(X_val)
        oof_preds.update(dict(zip(val_idx, val_preds)))
        
        test_preds[f'fold{fold}'] = model.predict(X_test)

        f1 = f1_score(y_val, val_preds)
        scores_f1.append(f1)
        if verbose:
            print(f'Fold #{fold}: ({model.best_iteration_} rounds) F1 = {f1:.5f}')
        
        _ = gc.collect()

    print(f'\nAvg F1 = {np.mean(scores_f1):.5f} +/- {np.std(scores_f1):.5f}')
    
    oof_preds = pd.Series(oof_preds).sort_index()
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0]

    return oof_preds, test_preds

In [19]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'objective': 'binary',
        'n_estimators': 10000,
        'boosting_type': 'goss',
        'extra_trees': True,
        'verbosity': -1,
        'device_type': 'cpu',
        'random_state': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='maximize'
    )
    print(f'Best trial: {study.best_trial.number} -> Best value(F1): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    oof_preds, test_preds = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params
    )
    return oof_preds, test_preds

In [20]:
oof_preds = pd.DataFrame()

### Generating submission files

In [21]:
NOTEBOOK = '03'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/zindi/landslide_prevention/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

In [22]:
def create_submission_files(test_preds: pd.DataFrame, expt_num: int):
    for col in (test_preds.columns):
        sub = sample_sub.copy()
        sub['Label'] = test_preds[col]
        sub.to_csv(f'{SUBMISSION_PATH}/{expt_num}_{col}.csv', index=False)

### Test experiment

In [23]:
%%time
o, t = run_experiment(
    data=(train, test, TARGET),
    n_trials=5
)

[I 2022-09-30 17:08:50,336] A new study created in memory with name: no-name-0241971e-93af-497d-8d81-38dee67fc827


---------------Hyperparameter tuning---------------


[I 2022-09-30 17:08:56,584] Trial 0 finished with value: 0.6204263574685877 and parameters: {'learning_rate': 0.16, 'reg_alpha': 189.4, 'reg_lambda': 153.1, 'num_leaves': 295, 'min_child_samples': 110, 'min_split_gain': 10.3, 'colsample_bytree': 0.55, 'top_rate': 0.25, 'other_rate': 0.35000000000000003, 'scale_pos_weight': 2.6500000000000004}. Best is trial 0 with value: 0.6204263574685877.
[I 2022-09-30 17:09:03,294] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.01, 'reg_alpha': 176.8, 'reg_lambda': 177.0, 'num_leaves': 315, 'min_child_samples': 294, 'min_split_gain': 14.68, 'colsample_bytree': 0.9, 'top_rate': 0.1, 'other_rate': 0.15000000000000002, 'scale_pos_weight': 2.1500000000000004}. Best is trial 0 with value: 0.6204263574685877.
[I 2022-09-30 17:09:12,664] Trial 2 finished with value: 0.6344832697055452 and parameters: {'learning_rate': 0.25, 'reg_alpha': 125.2, 'reg_lambda': 22.1, 'num_leaves': 20, 'min_child_samples': 472, 'min_split_gain': 2.12, 'col

Best trial: 3 -> Best value(F1): 0.64441
Best hyperparameters:
learning_rate        - 0.25
reg_alpha            - 143.6
reg_lambda           - 23.8
num_leaves           - 605
min_child_samples    - 64
min_split_gain       - 1.16
colsample_bytree     - 0.9
top_rate             - 0.30000000000000004
other_rate           - 0.1
scale_pos_weight     - 3.2
-----------------Cross-validation------------------
Fold #0: (69 rounds) F1 = 0.64387
Fold #1: (136 rounds) F1 = 0.64219
Fold #2: (166 rounds) F1 = 0.63709
Fold #3: (136 rounds) F1 = 0.64456
Fold #4: (365 rounds) F1 = 0.65436

Avg F1 = 0.64441 +/- 0.00562
CPU times: user 1min 36s, sys: 3.14 s, total: 1min 39s
Wall time: 56 s


### Experiment 1: Original features

In [ ]:
%%time
oof_preds['3_1'], test_preds_1 = run_experiment(
    data=(train, test, TARGET),
    n_trials=250
)

[I 2022-09-30 14:32:35,672] A new study created in memory with name: no-name-e6d69f06-8fbd-454b-be65-125c6e0a7274


---------------Hyperparameter tuning---------------


[I 2022-09-30 14:32:39,477] Trial 0 finished with value: 0.6204263574685877 and parameters: {'learning_rate': 0.16, 'reg_alpha': 189.4, 'reg_lambda': 153.1, 'num_leaves': 295, 'min_child_samples': 110, 'min_split_gain': 10.3, 'colsample_bytree': 0.55, 'top_rate': 0.25, 'other_rate': 0.35000000000000003, 'scale_pos_weight': 2.6500000000000004}. Best is trial 0 with value: 0.6204263574685877.
[I 2022-09-30 14:32:43,502] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.01, 'reg_alpha': 176.8, 'reg_lambda': 177.0, 'num_leaves': 315, 'min_child_samples': 294, 'min_split_gain': 14.68, 'colsample_bytree': 0.9, 'top_rate': 0.1, 'other_rate': 0.15000000000000002, 'scale_pos_weight': 2.1500000000000004}. Best is trial 0 with value: 0.6204263574685877.
[I 2022-09-30 14:32:49,467] Trial 2 finished with value: 0.6344832697055452 and parameters: {'learning_rate': 0.25, 'reg_alpha': 125.2, 'reg_lambda': 22.1, 'num_leaves': 20, 'min_child_samples': 472, 'min_split_gain': 2.12, 'col

Best trial: 218 -> Best value(F1): 0.70805
Best hyperparameters:
learning_rate        - 0.28
reg_alpha            - 0.30000000000000004
reg_lambda           - 160.9
num_leaves           - 660
min_child_samples    - 170
min_split_gain       - 0.0
colsample_bytree     - 0.6
top_rate             - 0.5
other_rate           - 0.3
scale_pos_weight     - 2.35
-----------------Cross-validation------------------
Fold #0: (302 rounds) F1 = 0.68346
Fold #1: (339 rounds) F1 = 0.71711
Fold #2: (195 rounds) F1 = 0.69697
Fold #3: (473 rounds) F1 = 0.71870
Fold #4: (205 rounds) F1 = 0.72403

Avg F1 = 0.70805 +/- 0.01535
CPU times: user 1h 8min 29s, sys: 47.9 s, total: 1h 9min 17s
Wall time: 36min 2s


In [ ]:
create_submission_files(test_preds_1, 1)

### Experiment 2: Created features

In [ ]:
created_features = [f for f in test_agg.columns 
                    if f.startswith('agg_') or f.startswith('grad_')]

In [ ]:
%%time
oof_preds['3_2'], test_preds_2 = run_experiment(
    data=(train_agg[created_features], test_agg[created_features], TARGET),
    n_trials=250
)

[I 2022-09-30 15:08:38,251] A new study created in memory with name: no-name-1a7c732a-312d-4e44-b55f-209385657abf


---------------Hyperparameter tuning---------------


[I 2022-09-30 15:08:46,292] Trial 0 finished with value: 0.6272752317531014 and parameters: {'learning_rate': 0.16, 'reg_alpha': 189.4, 'reg_lambda': 153.1, 'num_leaves': 295, 'min_child_samples': 110, 'min_split_gain': 10.3, 'colsample_bytree': 0.55, 'top_rate': 0.25, 'other_rate': 0.35000000000000003, 'scale_pos_weight': 2.6500000000000004}. Best is trial 0 with value: 0.6272752317531014.
[I 2022-09-30 15:08:52,180] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.01, 'reg_alpha': 176.8, 'reg_lambda': 177.0, 'num_leaves': 315, 'min_child_samples': 294, 'min_split_gain': 14.68, 'colsample_bytree': 0.9, 'top_rate': 0.1, 'other_rate': 0.15000000000000002, 'scale_pos_weight': 2.1500000000000004}. Best is trial 0 with value: 0.6272752317531014.
[I 2022-09-30 15:09:00,488] Trial 2 finished with value: 0.6531844934938589 and parameters: {'learning_rate': 0.25, 'reg_alpha': 125.2, 'reg_lambda': 22.1, 'num_leaves': 20, 'min_child_samples': 472, 'min_split_gain': 2.12, 'col

Best trial: 227 -> Best value(F1): 0.71832
Best hyperparameters:
learning_rate        - 0.25
reg_alpha            - 0.1
reg_lambda           - 192.70000000000002
num_leaves           - 450
min_child_samples    - 32
min_split_gain       - 0.02
colsample_bytree     - 0.55
top_rate             - 0.5
other_rate           - 0.15000000000000002
scale_pos_weight     - 3.25
-----------------Cross-validation------------------
Fold #0: (673 rounds) F1 = 0.68641
Fold #1: (532 rounds) F1 = 0.71675
Fold #2: (327 rounds) F1 = 0.70216
Fold #3: (266 rounds) F1 = 0.74629
Fold #4: (558 rounds) F1 = 0.73998

Avg F1 = 0.71832 +/- 0.02251
CPU times: user 1h 42min 31s, sys: 1min 1s, total: 1h 43min 32s
Wall time: 53min 49s


In [ ]:
create_submission_files(test_preds_2, 2)

### Experiment 3: All features

In [24]:
%%time
oof_preds['3_3'], test_preds_3 = run_experiment(
    data=(train_agg, test_agg, TARGET),
    n_trials=250
)

[I 2022-09-30 17:09:53,155] A new study created in memory with name: no-name-52997224-c404-4c96-8381-2307f21f8ca2


---------------Hyperparameter tuning---------------


[I 2022-09-30 17:10:07,099] Trial 0 finished with value: 0.6307715324886548 and parameters: {'learning_rate': 0.16, 'reg_alpha': 189.4, 'reg_lambda': 153.1, 'num_leaves': 295, 'min_child_samples': 110, 'min_split_gain': 10.3, 'colsample_bytree': 0.55, 'top_rate': 0.25, 'other_rate': 0.35000000000000003, 'scale_pos_weight': 2.6500000000000004}. Best is trial 0 with value: 0.6307715324886548.
[I 2022-09-30 17:10:21,198] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 0.01, 'reg_alpha': 176.8, 'reg_lambda': 177.0, 'num_leaves': 315, 'min_child_samples': 294, 'min_split_gain': 14.68, 'colsample_bytree': 0.9, 'top_rate': 0.1, 'other_rate': 0.15000000000000002, 'scale_pos_weight': 2.1500000000000004}. Best is trial 0 with value: 0.6307715324886548.
[I 2022-09-30 17:10:41,406] Trial 2 finished with value: 0.6507957391808563 and parameters: {'learning_rate': 0.25, 'reg_alpha': 125.2, 'reg_lambda': 22.1, 'num_leaves': 20, 'min_child_samples': 472, 'min_split_gain': 2.12, 'col

Best trial: 235 -> Best value(F1): 0.72920
Best hyperparameters:
learning_rate        - 0.16
reg_alpha            - 3.8000000000000003
reg_lambda           - 32.4
num_leaves           - 520
min_child_samples    - 4
min_split_gain       - 0.0
colsample_bytree     - 0.75
top_rate             - 0.2
other_rate           - 0.45
scale_pos_weight     - 3.1
-----------------Cross-validation------------------
Fold #0: (380 rounds) F1 = 0.70815
Fold #1: (167 rounds) F1 = 0.72604
Fold #2: (199 rounds) F1 = 0.70294
Fold #3: (563 rounds) F1 = 0.74781
Fold #4: (465 rounds) F1 = 0.76108

Avg F1 = 0.72920 +/- 0.02238
CPU times: user 4h 13min 26s, sys: 14min 43s, total: 4h 28min 10s
Wall time: 2h 28min 31s


In [25]:
create_submission_files(test_preds_3, 3)

### Storing OOF predictions

In [26]:
oof_preds.to_csv(f'{SUBMISSION_PATH}/oof_preds.csv', index=False)